In [ ]:
import pandas as pd

# กำหนดค่า year และ week_number
year = 2024
week_number = 23
file_path = 'G:/My Drive/usc_zonal_cir_3/' + str(year) + "-" + str(week_number) + '.csv'

# อ่านไฟล์ CSV เข้าไปใน DataFrame
df = pd.read_csv(file_path)

# คอลัมน์ที่สนใจสำหรับการวิเคราะห์ความสัมพันธ์
columns_of_interest = [ "usc", "co", "no2", "o3", "so2", "lst", "ndmi", "ndvi", "ndbi"]

# แสดงผล DataFrame ที่เลือกคอลัมน์ที่สนใจ
# print(df[columns_of_interest])

# กำหนดโฟลเดอร์ที่ใช้บันทึกไฟล์ Excel
output_folder = 'D:/py499/cir_3'

# สร้างชื่อไฟล์ Excel จากปีและเลขสัปดาห์
output_file = output_folder + f'/{year}-{week_number}.xlsx'

# บันทึก DataFrame เป็นไฟล์ Excel
df[columns_of_interest].to_excel(output_file, index=False)

print(f"DataFrame has been successfully saved to: {output_file}")






In [ ]:
# Import necessary libraries
import pandas as pd
import statsmodels.api as sm
# from google.colab import drive
from sklearn.model_selection import train_test_split

# Mount Google Drive
# drive.mount('/content/drive')

# กำหนดค่า year และ week_number
year = 2023
week_number = 5
file_path = 'G:/My Drive/usc_zonal_cir_2/' + str(year) + "-" + str(week_number) + '.csv'

# อ่านไฟล์ CSV เข้าไปใน DataFrame
df = pd.read_csv(file_path)


df = pd.read_csv(file_path)
df = df.dropna(subset= ["usc", "lst", "ndmi", "ndvi", "ndbi"])

if df.shape[0] == 0:
    raise ValueError("No samples available after dropping rows with null values.")

target_variable = 'co'
columns_of_interest =  [ "usc", "lst", "ndmi", "ndvi", "ndbi"]

X = df[columns_of_interest]
y = df[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Forward stepwise regression
def forward_stepwise_selection(X, y):
    initial_features = []
    remaining_features = list(X.columns)
    best_features = []
    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            features = initial_features + [candidate]
            X_train_with_candidate = sm.add_constant(X[features])
            model = sm.OLS(y, X_train_with_candidate).fit()
            score = model.rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort(reverse=True)
        best_new_score, best_candidate = scores_with_candidates[0]
        if initial_features:
            best_old_score = sm.OLS(y, sm.add_constant(X[initial_features])).fit().rsquared_adj
            if best_new_score <= best_old_score:
                break
        initial_features.append(best_candidate)
        remaining_features.remove(best_candidate)
        best_features.append((best_new_score, best_candidate))
    return best_features

# Perform forward stepwise selection
best_features = forward_stepwise_selection(X_train, y_train)

# Print the selected features and their scores
print("Selected features and their adjusted R-squared scores:")
for score, feature in best_features:
    print(f"{feature}: {score}")

# Build the final model with the selected features
selected_features = [feature for _, feature in best_features]
X_train_selected = sm.add_constant(X_train[selected_features])
X_test_selected = sm.add_constant(X_test[selected_features])
final_model = sm.OLS(y_train, X_train_selected).fit()

# Print the summary of the final model
print(final_model.summary())

# Evaluate the model on the test set
y_pred = final_model.predict(X_test_selected)
print(f"Test set R-squared: {final_model.rsquared}")
